#Run all cells and go to the end to try out your own sequences

###Install ESM and import dependencies

In [1]:
pip install fair-esm

In [2]:
import esm

In [58]:
import torch
import esm

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval().cuda()

def return_sequence_representation(sequences):
  data = [(f'peptide{i}', sequence) for i,sequence in enumerate(sequences)]
  batch_labels, batch_strs, batch_tokens = batch_converter(data)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

  with torch.no_grad():
      results = model(batch_tokens.cuda(), repr_layers=[33], return_contacts=True)
  token_representations = results["representations"][33]

  # Removing start and end tokens
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
    sequence_representations.append(token_representations[i, 1 : tokens_len - 1].cpu().numpy())

  return sequence_representations

In [40]:
import numpy as np
import pandas as pd
import glob
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LogisticRegressionCV, RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.metrics import auc, roc_auc_score
from sklearn import model_selection
from sklearn import metrics

In [41]:
pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.1 MB/s eta 0:00:00


In [42]:
from sktime.transformations.panel.rocket import Rocket

In [5]:
dimensions = return_sequence_representation(['GGGGGG']).shape[2]

In [6]:
dimensions

1280

###Load the CDP dataset and embeddings

In [7]:
# Load the cysteine-dense peptides dataset (CDPs.csv) from GitHub
df = pd.read_csv('https://raw.githubusercontent.com/Zebreu/cyspresso/main/CDPs.csv')

In [8]:
# Process the dataset (omit extraneous columns and label "Expressibility" as True or False)
express = {'+': True, '+-PR': True, '-': False}
df['Expressibility'] = df['Expressibility'].replace(express)

In [9]:
df = df[['Uniprot','Sequence','Expressibility','Is Knottin? Uniprot']]
df

,Uniprot,Sequence,Expressibility,Is Knottin? Uniprot
0,P01030,AKRCCQDGLTRLPMARTCEQRAARVQQPACREPFLSCCQFA,False,N
1,P46162,PQSCRWNMGVCIPFLCRVGMRQIGTCFGPRVPCCRR,False,N
2,P46163,PQSCRWNMGVCIPISCPGNMRQIGTCFGPRVPCCRRW,False,N
3,P46167,FVTCRINRGFCVPIRCPGHRRQIGTCLAPQIKCCR,False,N
4,P01223,GLACGQAMSFCIPTEYMMHVERKECAYCLTINTTVCAGYCMTR,False,N
...,...,...,...,...
1244,B6UHE2,ADLCVTRSRTFKGWCHQSENCITVCKSEGNTGGFCKLGACMCTKECVRS,True,N
1245,P0C1Y5,GGGCGYKDVNKAPFNSMGACGNVPIFKDGLGCGSCFEIKCDKPAECSGK,False,N
1246,B6SJ49,ARTCQSQSHRFRGPCLRRSNCANVCRTEGFPGGRCRGFRRRCFCTTHCH,False,N
1247,B6SQK6,AQICYSRSKTFKGWCYHSTNCISVCITEGEISGFCQHGICMCTYECLTG,False,N


In [14]:
# Avoids running out of memory on consumer GPUs
embeddings = []
for i in range(13):
  subset = df['Sequence'].values[i*100:(i+1)*100]
  embeddings.extend(return_sequence_representation(subset))

### Embedding preprocessing

In [25]:
# Rocket works best when zero-padded

features = [[] for i in range(dimensions)]
new_names = []
for name, embedding in zip(df['Uniprot'], embeddings):
  for i in range(dimensions):
    f = embedding[:,i]
    d = 50 - len(f) #Pad values with 0s to ensure equal length (50, in this case)"
    f = np.pad(f, (0,d))
    features[i].append(pd.Series(f))
  new_names.append(name)

In [ ]:
cc = pd.DataFrame(new_names, columns=['Uniprot'])
for i,f in enumerate(features):
  cc[i] = pd.Series(f)
  combined = pd.merge(cc, df, left_on='Uniprot', right_on='Uniprot')

In [ ]:
combined

In [36]:
# Omit duplicates from dataset
combined = combined.drop_duplicates('Uniprot').sort_values('Uniprot')
combined['Expressibility'].value_counts()

Expressibility
True     678
False    549
Name: count, dtype: int64

### Split the dataset between knottins and non-knottins if applicable

In [68]:
# Select all rows that correspond to knottin proteins based on Uniprot identification
if False: # Skipped for generalization given this notebook is designed to allow users to try out easily, but if you care about knottins then you train a more specific model using this subset
  knottin = combined[combined['Is Knottin? Uniprot'] == 'Y']
  knottin = knottin.sort_values('Uniprot')
  combined = knottin

###ROCKET transformation and model training

In [69]:
# ROCKET transformation on the knottin dataset

feat_cols = list(range(dimensions))

rocket = Rocket(num_kernels=10000, random_state=42)
rocket.fit(combined[feat_cols])
transformed = rocket.transform(combined[feat_cols])

In [70]:
# Train a logistic regression model
X_train = transformed.values
y_train = combined['Expressibility'].values

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
tr = scaler.transform(X_train)
regression_model = LogisticRegression(C=0.0001)
regression_model.fit(tr, y_train)

LogisticRegression(C=0.0001)

In [65]:
def predict(sequences):
  embeddings = return_sequence_representation(sequences)

  features = [[] for i in range(dimensions)]
  new_names = []
  for name, embedding in zip(sequences, embeddings):
    for i in range(dimensions):
      f = embedding[:,i]
      d = 50 - len(f) #Pad values with 0s to ensure equal length (50 being the maximum length in this case)"
      f = np.pad(f, (0,d))
      features[i].append(pd.Series(f))
    new_names.append(name)

  cc = pd.DataFrame(new_names, columns=['Sequences'])
  for i,f in enumerate(features):
    cc[i] = pd.Series(f)

  transformed = rocket.transform(cc[feat_cols])
  return regression_model.predict_proba(transformed)

#Bring your own sequences and get expression predictions

In [ ]:
sequences = [
  'PAPCVATRDSCKPPAPACCDPCASCQCRFFRSACSCRVLTRTC',
  'ERECLGFGKGCNPSNDQCCKSSNLVCSRKHRWCKYE'
]

predictions = predict(sequences)

In [98]:
predictions # the second column is the probability of expression

array([[0.69601175, 0.30398825],
       [0.10015277, 0.89984723]])